# 🎓 Parseval's Theorem

In this notebook, [Parseval's Theorem](https://en.wikipedia.org/w/index.php?title=Parseval%27s_theorem&oldid=1062705638#Notation_used_in_engineering) according to [Stull (1988)](http://dx.doi.org/10.1007/978-94-009-3027-8) is verified to be valid with PARMESAN's spectra.

In [ ]:
import matplotlib.dates
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(420)
import pandas as pd
import scipy.fft
import scipy.optimize
import scipy.signal

import parmesan

## ⚙ Generating a Signal

In [ ]:
n = 10000
sampling_rate = 9  # Hz
spectrum_kwargs = dict(window=None, detrend="linear")

In [ ]:
# times to use
seconds = np.arange(0, n, 1) / sampling_rate
timestamps = pd.to_datetime(seconds, unit="s", origin="now")

# generate a random walk signal as base
random_walk = np.cumsum((np.random.random(n) - 0.5) * 2)

# generate a wave with a much lower frequency than the sampling rate
wave_frequency = sampling_rate / 50
wave_amplitude = random_walk.var() / 100
wave = wave_amplitude * np.sin(2 * np.pi * wave_frequency * seconds)

df = pd.DataFrame(
    dict(signal=random_walk + wave),
    index=timestamps,
)
df.head()

## 🔧 Function to Plot Timeseries and Spectrum

In [ ]:
def plot_spectrum(**spectrum_kwargs):
    from numpy import abs, mean, sum

    plt.rcParams["figure.figsize"] = (10, 12)
    plt.rcParams["font.size"] = 13
    plt.rcParams["legend.labelspacing"] = 1.5
    plt.rcParams["legend.loc"] = "lower left"
    fig, (ax1, ax2) = plt.subplots(nrows=2)

    spectrum_kwargs_str = ", ".join(
        "{}={}".format(k, repr(v)) for k, v in spectrum_kwargs.items()
    )

    df.signal.plot(
        ax=ax1,
        label=f"random walk + ${wave_amplitude:.2f} \cdot \sin({wave_frequency:.2f}\mathrm{{Hz}})$\n"
        f"mean = {df.signal.mean():.2f}, variance $\sigma^2$ = {df.signal.var():.2f}",
    )

    (freq, power, blocks,) = parmesan.analysis.power_spectrum(
        x=seconds,
        y=df.signal.values,
        # we also want power_spectrum() to return the actual data it used for the FFT, so we instruct it to also return the "blocks"
        returnvalue=("frequency", "power", "blocks"),
        **spectrum_kwargs,
    )
    # put the conditioned data into the dataframe
    df["conditioned"] = blocks[0]  # we only have one block

    df.conditioned.plot(
        ax=ax1,
        label=f"conditioned signal ({spectrum_kwargs_str}) used in FFT\n"
        f"mean = {df.conditioned.mean():.2f}\n"
        f"$\\mathbf{{variance}}$ $\sigma^2 = \\mathbf{{{df.conditioned.var(ddof=0) = :.4f}}}$\n"
        f"$= \\mathtt{{{mean((df.conditioned-df.conditioned.mean())**2) = :.4f}}}$\n"
        f"signal energy = ${sum(abs(df.conditioned)**2) = :.1f}$",
    )

    ax1.set_title(r"Timeseries")
    ax1.legend()

    ax2.plot(
        freq,
        power,
        label=f"$\\mathtt{{parmesan.analysis.power\\_spectrum()}}$ of signal,\n"
        f"$\\mathbf{{variance}} = 2 \cdot \sum_{{i=1}}^{{{len(df.index)}}}\\left|X_{{FFT, i}}\\right|^2  "
        f"= \\mathtt{{\\mathbf{{{2 * power[1:].sum() = :.4f}}}}}$\n"
        # Integrate the mirrored spectrum (2*sum(power[1:])) and only count the constant coefficient
        f"signal energy = ${(power[0] + 2*sum(power[1:])) * (2 * power.size-2) = :.1f}$",
        linewidth=2,
        color="orange",
    )

    spectrum = df.signal.parmesan.spectrum(
        with_kolmogorov=True, **spectrum_kwargs
    )

    spectrum.filter(axis="columns", regex="^signal$").squeeze().plot(
        ax=ax2,
        label=f"$\\mathtt{{df.signal.parmesan.spectrum({spectrum_kwargs_str})}}$\nyields the same power spectrum",
        linestyle="dotted",
        alpha=0.5,
        color="green",
    )
    spectrum.filter(axis="columns", regex="power.*law").squeeze().plot(
        ax=ax2, color="black", linestyle="dashed"
    )

    ax2.axvline(
        sampling_rate,
        label=f"{sampling_rate = } Hz",
        linestyle="dotted",
        linewidth=5,
        color="black",
        alpha=0.2,
        zorder=-1,
    )
    ax2.axvline(
        wave_frequency,
        label=f"{wave_frequency = } Hz",
        linestyle="solid",
        linewidth=10,
        color="black",
        alpha=0.2,
        zorder=-1,
    )
    ax2.set_title("Power Spectrum")
    ax2.set_xscale("log")
    ax2.set_yscale("log")
    ax2.legend()

    fig.tight_layout()

## ✅ Verifying Parseval's Theorem

Note how in all the below plots the *variance* of the signal equals the *energy* in the frequency domain (the **bold** numbers in the legends below 👇).

According to Stull (1988) chapter 8 eq. 8.5a and 8.6.1b for a signal $A$ with $N$ sampling points:

$$
\mathrm{variance}
=
\sigma_\mathrm{A} ^ 2
=
\frac{1}{N} \sum_{k=0}^{N-1} \left( A_\mathrm{k} - \overline{A} \right) ^ 2
=
\sum_{k=1}^{N-1} \left| F_\mathrm{A}\left(k\right) \right| ^ 2
$$

Where $F_\mathrm{A}\left(n\right)$ is calculated from the signal $A$ divided by the amount of sampling points $N$.

<details>
<summary>On Wikipedia</summary>
    
The [Wikipedia Article detailing Parseval's Theorem](https://en.wikipedia.org/w/index.php?title=Parseval%27s_theorem&oldid=1062705638#Notation_used_in_engineering) puts it like this:

$$
\sum_{n=0}^{N-1} | x[n] |^2  = \frac{1}{N} \sum_{k=0}^{N-1} | X[k] |^2
$$

The left-hand side $\sum_{n=0}^{N-1} | x[n] |^2$ is the signal energy and divided by the sample size $N$ becomes the sample variance in case the signal averages to 0, i.e. the signal is just annomalies.
</details>

### Linear Detrending, then Hann Window

In [ ]:
plot_spectrum(detrend="linear", window="hann")

### Linear Detrending, then Tukey Window

In [ ]:
plot_spectrum(window="tukey", detrend="linear")

### Linear Detrending, no Window

In [ ]:
plot_spectrum(window=None, detrend="linear")

### No Detrending, no Window

In [ ]:
plot_spectrum(window=None, detrend=False)